# Data analysis of LexTALE dataset

In [63]:
# import all necessary modules
import pandas as pd
import os
import re

In [64]:
# read in all participant files
files = os.listdir()
ind = 1
for file in files:
    if file.endswith(".csv"):
        print(f'reading {file} now')
        reg_exp = re.compile('_\d\d')
        ppnr = file.split('_')[0]
        if ind ==1:
            data = pd.read_csv(file)
            data['Pp']= ppnr
        else:
            dummy = pd.read_csv(file)
            data = pd.concat([data,dummy], sort=True)
            data.Pp.iloc[-60:] = ppnr
        ind +=1

reading 1_data_LexTALE.csv now


In [65]:
# check the data
data.describe()

,trial_nr,wordstatus,response,rt,error
count,60.000000,60.000000,60.000000,60.000000,60.000000
mean,30.500000,0.666667,0.716667,0.524132,0.183333
std,17.464249,0.475383,0.454420,0.188055,0.390205
min,1.000000,0.000000,0.000000,0.280221,0.000000
25%,15.750000,0.000000,0.000000,0.411076,0.000000
50%,30.500000,1.000000,1.000000,0.468974,0.000000
75%,45.250000,1.000000,1.000000,0.601845,0.000000
max,60.000000,1.000000,1.000000,1.484523,1.000000


In [66]:
# calculate scores per participant and put them in a dataframe 

In [ ]:
# make a histogram of the scores

In [ ]:
# check mean RT for nonwords and words 

In [ ]:
# check mean RT for nonwords and words, when only looking at hits and correct rejections

In [ ]:
# run a linear model to test whether RTs differ for words and nonwords